In [1]:
# Cell 1: Import Libraries
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Constants
DATA_FOLDER = 'final_data'
IMG_SIZE = (224, 224)  # Adjust based on ResNet50 requirements
BATCH_SIZE = 32  # Can be adjusted based on your system's capabilities

# Cell 2: Prepare Data
def prepare_data(data_folder, img_size, batch_size):
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input, validation_split=0.2)

    train_generator = datagen.flow_from_directory(
        data_folder,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        data_folder,
        target_size=img_size,
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=False  # Important for correct label ordering in evaluation
    )
    
    return train_generator, validation_generator

# Cell 3: Set Up ResNet50 Model for Transfer Learning
def setup_model(train_data, img_size):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=img_size + (3,))
    base_model.trainable = False  # Freeze base model layers

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)  # Adjust the size of the dense layer if needed
    predictions = Dense(train_data.num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)
    return model

# Cell 4: Compile and Train the Model
def compile_and_train_model(model, train_data, val_data):
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    history = model.fit(
        train_data,
        steps_per_epoch=train_data.samples // train_data.batch_size,
        validation_data=val_data,
        validation_steps=val_data.samples // val_data.batch_size,
        epochs=10  # Adjust the number of epochs
    )
    return history

# Cell 5: Evaluate the Model
def evaluate_model(model, val_data):
    val_predictions = model.predict(val_data)
    val_labels = np.argmax(val_predictions, axis=1)
    true_labels = val_data.classes
    class_labels = list(val_data.class_indices.keys())

    print("Confusion Matrix")
    print(confusion_matrix(true_labels, val_labels))

    print("\nClassification Report")
    print(classification_report(true_labels, val_labels, target_names=class_labels))

# Cell 6: Main Function to Run the Workflow
def main():
    train_generator, validation_generator = prepare_data(DATA_FOLDER, IMG_SIZE, BATCH_SIZE)
    model = setup_model(train_generator, IMG_SIZE)
    compile_and_train_model(model, train_generator, validation_generator)
    evaluate_model(model, validation_generator)

if __name__ == "__main__":
    main()


Found 1061 images belonging to 6 classes.
Found 263 images belonging to 6 classes.
Epoch 1/10
33/33 [==============================] - 89s 3s/step - loss: 2.6210 - accuracy: 0.3032 - val_loss: 1.5362 - val_accuracy: 0.3984
Epoch 2/10
33/33 [==============================] - 78s 2s/step - loss: 1.4729 - accuracy: 0.3829 - val_loss: 1.5202 - val_accuracy: 0.3945
Epoch 3/10
33/33 [==============================] - 78s 2s/step - loss: 1.3513 - accuracy: 0.4742 - val_loss: 1.4024 - val_accuracy: 0.4258
Epoch 4/10
33/33 [==============================] - 78s 2s/step - loss: 1.2927 - accuracy: 0.4830 - val_loss: 1.3377 - val_accuracy: 0.4648
Epoch 5/10
33/33 [==============================] - 78s 2s/step - loss: 1.1735 - accuracy: 0.5471 - val_loss: 1.3351 - val_accuracy: 0.4648
Epoch 6/10
33/33 [==============================] - 78s 2s/step - loss: 1.1713 - accuracy: 0.5442 - val_loss: 1.2832 - val_accuracy: 0.4805
Epoch 7/10
33/33 [==============================] - 89s 3s/step - loss: 1.089

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import vgg16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Constants
DATA_FOLDER = 'final_data'  # Directory with preprocessed and augmented data
IMG_SIZE = (224, 224)  # VGG16 standard image size
BATCH_SIZE = 32

# Function for Preparing Data
def prepare_data():
    # Preprocessing the input 
    datagen = ImageDataGenerator(preprocessing_function=vgg16.preprocess_input, validation_split=0.2)

    # Prepare generators
    train_generator = datagen.flow_from_directory(
        DATA_FOLDER,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        DATA_FOLDER,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Function to Build and Compile the VGG16 Model
def build_vgg16_model(train_data):
    # Load the VGG16 model, pre-trained on ImageNet data
    vgg16_base = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))

    # Freeze the layers
    for layer in vgg16_base.layers:
        layer.trainable = False

    # Add custom layers on top for our specific task
    x = vgg16_base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)  # Dropout added
    predictions = Dense(train_data.num_classes, activation='softmax')(x)

    # Compile the model
    model = Model(inputs=vgg16_base.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to Train and Evaluate the Model
def train_and_evaluate(model, train_generator, validation_generator):
    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE,
        epochs=10  # Adjust as needed
    )

    # Evaluate the model
    val_predictions = model.predict(validation_generator)
    val_labels = np.argmax(val_predictions, axis=1)
    true_labels = validation_generator.classes
    class_labels = list(validation_generator.class_indices.keys())

    print("Confusion Matrix")
    print(confusion_matrix(true_labels, val_labels))

    print("\nClassification Report")
    print(classification_report(true_labels, val_labels, target_names=class_labels))

# Main execution
def main():
    train_generator, validation_generator = prepare_data()
    vgg16_model = build_vgg16_model(train_generator)
    train_and_evaluate(vgg16_model, train_generator, validation_generator)

if __name__ == '__main__':
    main()


Found 1061 images belonging to 6 classes.
Found 263 images belonging to 6 classes.
Epoch 1/10
33/33 [==============================] - 447s 13s/step - loss: 3.7095 - accuracy: 0.2653 - val_loss: 1.5615 - val_accuracy: 0.3828
Epoch 2/10
33/33 [==============================] - 350s 11s/step - loss: 2.1309 - accuracy: 0.3392 - val_loss: 1.3818 - val_accuracy: 0.4570
Epoch 3/10
33/33 [==============================] - 273s 8s/step - loss: 1.5581 - accuracy: 0.4004 - val_loss: 1.3339 - val_accuracy: 0.4570
Epoch 4/10
33/33 [==============================] - 257s 8s/step - loss: 1.3439 - accuracy: 0.4684 - val_loss: 1.4314 - val_accuracy: 0.3711
Epoch 5/10
33/33 [==============================] - 226s 7s/step - loss: 1.2918 - accuracy: 0.4781 - val_loss: 1.3451 - val_accuracy: 0.4219
Epoch 6/10
33/33 [==============================] - 243s 7s/step - loss: 1.2186 - accuracy: 0.4927 - val_loss: 1.3042 - val_accuracy: 0.4531
Epoch 7/10
33/33 [==============================] - 240s 7s/step - lo

In [1]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import nasnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Constants
DATA_FOLDER = 'final_data'  # Directory with preprocessed and augmented data
IMG_SIZE = (224, 224)  # Adjust as per NASNetMobile requirements
BATCH_SIZE = 32

# Function for Preparing Data
def prepare_data():
    # Preprocessing the input 
    datagen = ImageDataGenerator(preprocessing_function=nasnet.preprocess_input, validation_split=0.2)

    # Prepare generators
    train_generator = datagen.flow_from_directory(
        DATA_FOLDER,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        DATA_FOLDER,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        subset='validation'
    )
    return train_generator, validation_generator

# Function to Build and Compile the NASNetMobile Model
def build_nasnet_model(train_data):
    # Load the NASNetMobile model, pre-trained on ImageNet data
    nasnet_base = nasnet.NASNetMobile(weights='imagenet', include_top=False, input_shape=IMG_SIZE + (3,))

    # Freeze the layers
    for layer in nasnet_base.layers:
        layer.trainable = False

    # Add custom layers on top for our specific task
    x = nasnet_base.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)  # Dropout added
    predictions = Dense(train_data.num_classes, activation='softmax')(x)

    # Compile the model
    model = Model(inputs=nasnet_base.input, outputs=predictions)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Function to Train and Evaluate the Model
def train_and_evaluate(model, train_generator, validation_generator):
    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // BATCH_SIZE,
        validation_data=validation_generator,
        validation_steps=validation_generator.samples // BATCH_SIZE,
        epochs=10  # Adjust as needed
    )

    # Evaluate the model
    val_predictions = model.predict(validation_generator)
    val_labels = np.argmax(val_predictions, axis=1)
    true_labels = validation_generator.classes
    class_labels = list(validation_generator.class_indices.keys())

    print("Confusion Matrix")
    print(confusion_matrix(true_labels, val_labels))

    print("\nClassification Report")
    print(classification_report(true_labels, val_labels, target_names=class_labels))

# Main execution
def main():
    train_generator, validation_generator = prepare_data()
    nasnet_model = build_nasnet_model(train_generator)
    train_and_evaluate(nasnet_model, train_generator, validation_generator)

if __name__ == '__main__':
    main()


Found 1061 images belonging to 6 classes.
Found 263 images belonging to 6 classes.
19993432/19993432 [==============================] - 7s 0us/step
Epoch 1/10
33/33 [==============================] - 50s 1s/step - loss: 1.9343 - accuracy: 0.3071 - val_loss: 1.6076 - val_accuracy: 0.3242
Epoch 2/10
33/33 [==============================] - 39s 1s/step - loss: 1.5758 - accuracy: 0.3790 - val_loss: 1.4976 - val_accuracy: 0.4180
Epoch 3/10
33/33 [==============================] - 40s 1s/step - loss: 1.5465 - accuracy: 0.3878 - val_loss: 1.5464 - val_accuracy: 0.3203
Epoch 4/10
33/33 [==============================] - 39s 1s/step - loss: 1.4421 - accuracy: 0.4344 - val_loss: 1.5556 - val_accuracy: 0.3242
Epoch 5/10
33/33 [==============================] - 39s 1s/step - loss: 1.4730 - accuracy: 0.3955 - val_loss: 1.4709 - val_accuracy: 0.3828
Epoch 6/10
33/33 [==============================] - 39s 1s/step - loss: 1.4121 - accuracy: 0.4325 - val_loss: 1.4411 - val_accuracy: 0.4531
Epoch 7/10
3